In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)

In [2]:
public_df = pd.read_csv('../data/inn_info_public.csv', dtype={'okved2': 'str', 'region': 'str'})
private_df = pd.read_csv('../data/inn_info_private.csv', dtype={'okved2': 'str'})
private_df['okved2'] = '1000'
pays_df = pd.read_csv('../data/pays.csv')

In [3]:
unknown_targets = public_df[public_df['is_public']==False][['hash_inn', 'region']]

In [4]:
unknown_targets = pd.merge(unknown_targets, 
                           private_df, 
                           on = 'hash_inn', 
                           how = 'left')

In [5]:
categotical_df = pd.concat([public_df[public_df['is_public']==True][['hash_inn', 'okved2', 'region']], 
                           unknown_targets[['hash_inn', 'okved2', 'region']]
                           ], axis = 0)

In [6]:
#возьмем 1000 случайных сэмплов
categotical_df = categotical_df.sample(1000, replace=False)

In [7]:
def create_pivot(pays_df, categotical_df, 
                 direction: str,
                 groupby_col: str,
                 agg_col: str,
                 cnt_not_null_cols = 1):
    """
    :direction - направление платежа: kt - входящий, dt - исходящий
    :groupby_col - категорий, по которой группируем (okved2 или region)
    :agg_col - колонка для группировки 
    :cnt_not_null_cols = 0 - количество колонок с ненулевым результатом после пивота
    """
            
    if direction == 'to':
        inn = 'hash_inn_dt'
        postfix = '_kt'
    elif direction == 'from':
        inn = 'hash_inn_kt'
        postfix = '_dt'
        
    pays_df = pd.merge(pays_df, categotical_df,
                       left_on = inn,
                       right_on = 'hash_inn',
                       how = 'left')

    pays_df = pd.pivot_table(pays_df, 
                             values=agg_col, 
                             index='hash_inn'+postfix,
                             columns=[groupby_col],
                             aggfunc='sum')
    pays_df.columns = [int(i) for i in pays_df.columns]
    cols_list = [col for col in pays_df.columns if col not in ['total', 'cnt_not_null']]
    
    pays_df['cnt_not_null'] = pays_df[cols_list].count(axis=1)
    pays_df = pays_df[pays_df['cnt_not_null']>cnt_not_null_cols]
    print('len of data: {}'.format(pays_df.shape[0]))

    for col in pays_df[cols_list]:
        pays_df.rename(columns={col: str(col)+'_'+agg_col+postfix}, inplace=True)
        
    for col in pays_df.columns:
        if '_kt' not in col: 
            pays_df.rename(columns={col: col+postfix}, inplace=True)

    pays_df = pays_df \
                .reset_index(drop=False) \
                .rename(columns={'hash_inn'+postfix: 'hash_inn'})
    return pays_df

In [8]:
%%time
df_to = create_pivot(pays_df, categotical_df, 
                 direction='to',
                 agg_col='count',
                 groupby_col = 'okved2',
                 cnt_not_null_cols=0)

len of data: 4259
CPU times: user 1.11 s, sys: 301 ms, total: 1.41 s
Wall time: 1.41 s


In [9]:
%%time
df_from = create_pivot(pays_df, categotical_df, 
                 direction='from',
                 agg_col='count',
                 groupby_col = 'okved2',
                 cnt_not_null_cols=0)

len of data: 6724
CPU times: user 969 ms, sys: 271 ms, total: 1.24 s
Wall time: 1.24 s


In [10]:
%%time
result_df = pd.merge(categotical_df, df_to,
                       on='hash_inn',
                       how='left')

result_df = pd.merge(result_df, df_from,
                       on='hash_inn',
                       how='left')

CPU times: user 8.56 ms, sys: 11.4 ms, total: 20 ms
Wall time: 19.2 ms


In [11]:
result_df.head(2)

,hash_inn,okved2,region,0_count_kt,10_count_kt,1000_count_kt,11_count_kt,12_count_kt,14_count_kt,15_count_kt,16_count_kt,17_count_kt,18_count_kt,20_count_kt,21_count_kt,22_count_kt,23_count_kt,24_count_kt,26_count_kt,28_count_kt,29_count_kt,3_count_kt,30_count_kt,32_count_kt,34_count_kt,37_count_kt,38_count_kt,39_count_kt,4_count_kt,41_count_kt,42_count_kt,43_count_kt,44_count_kt,45_count_kt,46_count_kt,47_count_kt,48_count_kt,50_count_kt,52_count_kt,53_count_kt,54_count_kt,55_count_kt,56_count_kt,57_count_kt,59_count_kt,6_count_kt,60_count_kt,61_count_kt,62_count_kt,65_count_kt,66_count_kt,67_count_kt,68_count_kt,69_count_kt,70_count_kt,71_count_kt,74_count_kt,76_count_kt,77_count_kt,79_count_kt,8_count_kt,9_count_kt,cnt_not_null_kt,0_count_dt_dt,10_count_dt_dt,1000_count_dt_dt,11_count_dt_dt,12_count_dt_dt,14_count_dt_dt,15_count_dt_dt,16_count_dt_dt,18_count_dt_dt,20_count_dt_dt,21_count_dt_dt,22_count_dt_dt,23_count_dt_dt,24_count_dt_dt,26_count_dt_dt,28_count_dt_dt,29_count_dt_dt,3_count_dt_dt,31_count_dt_dt,32_count_dt_dt,33_count_dt_dt,34_count_dt_dt,37_count_dt_dt,38_count_dt_dt,39_count_dt_dt,4_count_dt_dt,41_count_dt_dt,42_count_dt_dt,43_count_dt_dt,44_count_dt_dt,45_count_dt_dt,46_count_dt_dt,47_count_dt_dt,48_count_dt_dt,5_count_dt_dt,50_count_dt_dt,51_count_dt_dt,52_count_dt_dt,53_count_dt_dt,54_count_dt_dt,55_count_dt_dt,56_count_dt_dt,57_count_dt_dt,6_count_dt_dt,60_count_dt_dt,61_count_dt_dt,62_count_dt_dt,65_count_dt_dt,68_count_dt_dt,69_count_dt_dt,7_count_dt_dt,70_count_dt_dt,71_count_dt_dt,74_count_dt_dt,76_count_dt_dt,77_count_dt_dt,78_count_dt_dt,79_count_dt_dt,8_count_dt_dt,9_count_dt_dt,cnt_not_null_dt
0,93335,12,82,NaN,NaN,NaN,NaN,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,108.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,147650,12,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
result_df.shape

(1000, 124)

In [13]:
result_df[(result_df['cnt_not_null_kt'].isna()) & (result_df['cnt_not_null_dt'].isna())].shape

(893, 124)

In [14]:
result_df.to_csv('../data/item_result_df.csv', index=False, header=True)